# Solving an Income Fluctuation Problem with Collocation

Victoria Gregory

July 13, 2017

\begin{align*}
V(a, y) = \max_{c, a'} u(c) + \beta \displaystyle \sum_{y' \in Y} \pi(y, y') V(a', y') 
\end{align*}
subject to:
\begin{align*}
c + a' &= (1+r)a + y \\
a' &\geq 0
\end{align*}

Let the utility be function be log: $u(c) = \log(c)$

Let income, $y$, take on two values $[0.5 \hspace{1mm} 1.5]'$ with transition matrix 
$\pi(y, y') = \begin{bmatrix}
0.67 & 0.33 \\
0.33 & 0.67
\end{bmatrix}$

In [37]:
using QuantEcon
using BasisMatrices
using Plots
plotlyjs()

Plots.PlotlyJSBackend()

Define and set up the type that defines the household's problem.

In [38]:
type Household

    # User-inputted
    r::Float64              # interest rate
    β::Float64              # discount rate
    y_chain::MarkovChain{Float64, Matrix{Float64}, Vector{Float64}}     # Markov chain for income
    amin::Float64           # lower bound on asset grid
    amax::Float64           # upper bound on asset grid
    asize::Int64            # number of points on asset grid
    curv::Float64           # curvature of asset grid
    order::Int64            # order of splines 

    # Generated in constructor
    ygrid::Vector{Float64}  # income grid
    agrid::Vector{Float64}  # asset grid
    s::Matrix{Float64}      # full state space
    Ny::Int64               # number of points on income grid
    Na::Int64               # final number of points on asset grid
    Ns::Int64               # number of points on state space
    basis::Basis            # main Basis type
    bs::BasisMatrix         # main basis matrix (Direct)
    Φ::SparseMatrixCSC      # main basis matric (Expanded)
    Emat::SparseMatrixCSC   # transition matrix times basis matrix
    Φy::SparseMatrixCSC     # income part of the basis matrix

    # Solution
    c1::Vector{Float64}     # coefficients on value function
    c2::Vector{Float64}     # coefficients on expected value function
    ap::Vector{Float64}     # asset policy function

end

function Household(;r::Float64=0.03, β::Float64=0.96,
                   y_chain::MarkovChain{Float64,Matrix{Float64},Vector{Float64}}=MarkovChain([0.67 0.33; 0.33 0.67], [0.5; 1.5]), 
                   amin::Float64=1e-10, amax::Float64=20.0, asize::Int64=100,
                   curv::Float64=0.4, order::Int64=3)

    # State space for idiosyncratic shocks
    Π = y_chain.p
    ygrid0 = y_chain.state_values

    # State space for assets (endogenous)
    agrid0 = linspace(amin^curv, amax^curv, asize).^(1/curv)

    # Define the basis over the state variables
    basis = Basis(SplineParams(agrid0, 0, order),
                  LinParams(ygrid0, 0))
    s, (agrid, ygrid) = nodes(basis)
    Ns, Na, Ny = size(s, 1), size(agrid, 1), size(ygrid, 1)

    # Compute the basis matrix and expectations matrix
    bs = BasisMatrix(basis, Direct(), s, [0 0])
    Φ = convert(Expanded, bs).vals[1]
    Emat = kron(Π, speye(Na))*Φ

    # Save just the y part of the interpolation
    Φy = bs.vals[2] 

    # Initial guess for coefficients on value and expected value function
    c1 = zeros(Ns, )
    c2 = zeros(Ns, )

    # Placeholder for policy function
    ap = zeros(Ns, )
    
    return Household(r, β, y_chain, amin, amax, asize, curv, order, 
    ygrid, agrid, s, Ny, Na, Ns, basis, bs, Φ, Emat, Φy, c1, c2, ap)

end


Household

Given the state space, `s`, and the current policy function, `ap`, the following functions compute utility and the entire right-hand side of the Bellman equation.

In [39]:
function utility(h::Household, s::Matrix{Float64}, ap::Vector{Float64})

    c = (1 + h.r).*s[:, 1] + s[:, 2] - ap
    u = zeros(c)
    for cc=1:length(c)
        if c[cc] <= 0
            u[cc] = -10000000
        else
            u[cc] = log(c[cc])
        end
    end
    return u

end

function value(h::Household, s::Matrix{Float64}, 
               ap::Vector{Float64}; other_pols::Bool=true)

    # Compute flow payoff
    u = utility(h, s, ap)

    # Basis matrix for continuation value
    bs_a = BasisMatrix(h.basis[1], Direct(), ap, 0)
    Φa = bs_a.vals[1]
    Φapy = row_kron(h.Φy, Φa)

    # Compute value
    v1 = u + h.β*Φapy*h.c2

    if other_pols
        return v1, Φapy
    else
        return v1
    end

end

value (generic function with 1 method)

Solve for the optimal policy and compute both the value function and the expected value function, as well as the Jacobian.

In [40]:
function opt_value(h::Household, s::Matrix{Float64}; compute_jac::Bool=false)

    # Solve maximization problem
    lower_bound = zeros(size(s, 1), )
    upper_bound = (1 + h.r).*s[:, 1] + s[:, 2]
    f(ap::Vector{Float64}) = value(h, s, ap; other_pols=false)
    ap, v1 = golden_method(f, lower_bound, upper_bound)
    v1, Φapy = value(h, s, ap; other_pols=true)

    # Compute expected value function
    v2 = h.Emat*h.c1

    jac = []
    if compute_jac
        jac = [h.Φ -h.β*Φapy; -h.Emat h.Φ]
    end

    return v1, v2, ap, Φapy, jac

end

opt_value (generic function with 1 method)

The rest of the functions implement the Bellman iterations and the Newton iterations.

In [41]:
"""
One iteration on the Bellman equation
"""
function bellman_update!(h::Household)

    # Compute values
    s = h.s
    v1, v2, ap = opt_value(h, s)

    # Update coefficients
    c1 = h.Φ\v1
    c2 = h.Φ\v2
    h.c1 = c1
    h.c2 = c2
    h.ap = ap

    Void

end

"""
One Newton iteration on the problem.
"""
function newton_update!(h::Household)

  # Compute values
  s = h.s
  v1, v2, ap, Φapy, jac = opt_value(h, s; compute_jac=true)

  # Update coefficients
  cold = [h.c1; h.c2]
  c = cold - jac\([h.Φ*h.c1 - v1; h.Φ*h.c2 - v2])
  h.c1 = c[1:h.Ns]
  h.c2 = c[h.Ns+1:2*h.Ns]
  h.ap = ap

  Void

end

"""
Solve the problem via VFI
"""
function vfi!(h::Household; tol::Float64=1e-8, print::Bool=true, 
              Nbell::Int64=3, Nnewt::Int64=50)

    # Bellman iterations: set number of times
    i = 0
    dc = 1
    while dc > tol && i < Nbell
        i = i + 1
        c_old = [h.c1; h.c2]
        bellman_update!(h)
        c_new = [h.c1; h.c2]
        dc = maxabs(c_new - c_old)
        if print
            println("Bellman iteration $(i); distance of $(dc)")
        end
    end

    # Newton iterations: until convergence
    i = 0
    while i < Nnewt && dc > tol
        i = i + 1
        cold = [h.c1; h.c2]
        newton_update!(h)
        cnew = [h.c1; h.c2]
        dc = maxabs(cnew - cold)
        if print
            println("Newton iteration $(i); distance of $(dc)")
        end
    end

    Void

end

vfi!

Finally, a function to create some simple plots of the decision rules.

In [42]:
function plot_policies(h::Household)

    ap = reshape(h.ap, h.Na, h.Ny)
    c = (1 + h.r).*h.s[:, 1] + h.s[:, 2] - h.ap
    c = reshape(c, h.Na, h.Ny)

    labs = reshape(["Low Income", "High Income"], 1, h.Ny)
    plt_a = plot(h.agrid, ap, label=labs, title="Assets Tomorrow")
    plt_c = plot(h.agrid, c, label=labs, title="Consumption")
    plt = plot(plt_a, plt_c, layout=(1, 2), lw=2, 
               xlabel="Assets Today", size=(900, 500))
    return plt

end

plot_policies (generic function with 1 method)

Let's see how the code works. Below, I create an instance of the `Household` type with the default parameters, solve the problem, and plot the resulting policy functions.

In [43]:
h = Household()
@time vfi!(h)

Bellman iteration 1; distance of 3.095577608523707
Bellman iteration 2; distance of 3.0802971010699505
Bellman iteration 3; distance of 1.7394219214891282
Newton iteration 1; distance of 2.3575470852641676
Newton iteration 2; distance of 4.703222487804352
Newton iteration 3; distance of 2.32861682515691
Newton iteration 4; distance of 0.4805637796459674
Newton iteration 5; distance of 0.018374462486137944
Newton iteration 6; distance of 2.4239809802040213e-5
Newton iteration 7; distance of 7.622347197866475e-11
  0.427676 seconds (142.75 k allocations: 132.525 MB, 7.25% gc time)


Void

In [44]:
plot_policies(h)

In [45]:
h = Household()
@time vfi!(h; Nbell=100)

Bellman iteration 1; distance of 3.095577608523707
Bellman iteration 2; distance of 3.0802971010699505
Bellman iteration 3; distance of 1.7394219214891282
Bellman iteration 4; distance of 1.7165544892312585
Bellman iteration 5; distance of 1.266484996028585
Bellman iteration 6; distance of 1.2463209041193668
Bellman iteration 7; distance of 0.9845013612649263
Bellman iteration 8; distance of 0.9675192721951067
Bellman iteration 9; distance of 0.7932130890268176
Bellman iteration 10; distance of 0.7786333809924235
Bellman iteration 11; distance of 0.6541934230114173
Bellman iteration 12; distance of 0.6414012522037789
Bellman iteration 13; distance of 0.548564135890075
Bellman iteration 14; distance of 0.53717851570668
Bellman iteration 15; distance of 0.46575526677078827
Bellman iteration 16; distance of 0.4555355241106458
Bellman iteration 17; distance of 0.39931331080446597
Bellman iteration 18; distance of 0.39009261001036144
Bellman iteration 19; distance of 0.3450393349545706
Bell

Void